# Random SK Instances

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## Data set overview plots

### Small/Large-gap data

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# N = 19
# pattern = r"random_SK_instance_N_19_seed_(\d+)\.h5"

In [ ]:
# final time for mean-field
# T_final = 10000.
T_final = 32000.

# number of points to get Lyapunov exponent for
npts = 256
npts = 2048

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
subdir = "small_gaps"
# subdir = "late_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("most_undecided_spin", x), instance_names)

couplings_small_gap = Dict()
eigvals_small_gap = Dict()
lyapunov_exponents_small_gap = Dict()
bogo_spec_small_gap = Dict()
most_undecided_spins_small = Dict(zip(h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6))),
                                      h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)))))
most_undecided_flucs_small = Dict()

most_undecided_spins = Dict()

for instance_name in instance_names#[2000:2002]
    seed = match(pattern, instance_name)[1]
    if occursin("results", instance_name)
        try
            # # Mean-field trajectories
            # sol_t = h5read(folder_name * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", 32768., log10(1e-6)))
            # sol_u = h5read(folder_name * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", 32768., log10(1e-6)))
            # nzs = reduce(hcat, [sol_u[k, 3, :] for k in 1:size(sol_u)[1]])

            # # Get "most undecided spin" from area under z components
            # areas = Dict()
            # dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
            # for spin_idx in 1:N-1
            #     areas[spin_idx] = sum(dts .* nzs[spin_idx, 2:end]) |> abs
            # end
            # undecided_spins = [k for (k, v) in sort(areas |> collect, by=x->x[2])]
            # most_undecided_spin = undecided_spins[1]
            # most_undecided_spins[seed] = most_undecided_spin

            # Fluctuations
            # lyapunov_exponents_small_gap[seed] = h5read(folder_name * instance_name, @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
            # all_flucs = sum(h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts)), dims=1)
            all_flucs = h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
            lyapunov_exponent = sum(all_flucs, dims=1)
            if sum(lyapunov_exponent) |> abs < 1e4 # discard non-converged ones
                lyapunov_exponents_small_gap[seed] = lyapunov_exponent
                most_undecided_flucs_small[seed] = all_flucs[most_undecided_spins_small[seed], :]
            end

            # Bogoliubov spectrum
            bogo_spec = h5read(folder_name * instance_name, @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32))
            bogo_spec_small_gap[seed] = bogo_spec

        catch e
            printstyled(seed, ": ", e, "\n", color=:red)
        end            
        continue
    end
    try    
        couplings = h5read(folder_name * instance_name, "J")
        couplings_small_gap[seed] = couplings
        eigvals_small_gap[seed] = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end
end

In [ ]:
subdir = "large_gaps"
# subdir = "extra_data"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
instance_names = readdir(folder_name)
filter!(x -> !occursin("most_undecided_spin", x), instance_names);

couplings_large_gap = Dict()
eigvals_large_gap = Dict()
lyapunov_exponents_large_gap = Dict()
bogo_spec_large_gap = Dict()
most_undecided_spins_large = Dict(zip(h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6))),
                                      h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)))))
most_undecided_flucs_large = Dict()                                      

most_undecided_spins = Dict()

for instance_name in instance_names
    seed = match(pattern, instance_name)[1]
    if occursin("results", instance_name)
        # # Mean-field trajectories
        # sol_t = h5read(folder_name * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", 32768., log10(1e-6)))
        # sol_u = h5read(folder_name * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", 32768., log10(1e-6)))
        # nzs = reduce(hcat, [sol_u[k, 3, :] for k in 1:size(sol_u)[1]])

        # # Get "most undecided spin" from area under z components
        # areas = Dict()
        # dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
        # for spin_idx in 1:N-1
        #     areas[spin_idx] = sum(dts .* nzs[spin_idx, 2:end]) |> abs
        # end
        # undecided_spins = [k for (k, v) in sort(areas |> collect, by=x->x[2])]
        # most_undecided_spin = undecided_spins[1]
        # most_undecided_spins[seed] = most_undecided_spin        

        # Fluctuations
        # lyapunov_exponents_large_gap[seed] = h5read(folder_name * "results_" * instance_name, @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))    
        # lyapunov_exponents_large_gap[seed] = sum(h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts)), dims=1)
        all_flucs = h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
        lyapunov_exponent = sum(all_flucs, dims=1)
        if sum(lyapunov_exponent) |> abs < 1e4 # discard non-converged ones
            lyapunov_exponents_large_gap[seed] = lyapunov_exponent
            most_undecided_flucs_large[seed] = all_flucs[most_undecided_spins_large[seed], :]
        end

        # Bogoliubov spectrum
        bogo_spec = h5read(folder_name * instance_name, @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32))
        bogo_spec_large_gap[seed] = bogo_spec
        continue
    end
    try
        couplings = h5read(folder_name * instance_name, "J")
        couplings_large_gap[seed] = couplings
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        eigvals_large_gap[seed] = λ
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end                                             
end

In [ ]:
# h5write(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)), [k for (k, v) in most_undecided_spins])

# h5write(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), 
# @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)), [v for (k, v) in most_undecided_spins])

In [ ]:
eigvals_small = [v[2, :] .- v[1, :] for (k, v) in eigvals_small_gap]
eigvals_large = [v[2, :] .- v[1, :] for (k, v) in eigvals_large_gap]
avg_eigvals_small = eigvals_small |> mean
avg_eigvals_large = eigvals_large |> mean;

In [ ]:
avg_lyapunov_exponents_small = [v for (k, v) in lyapunov_exponents_small_gap] |> mean
avg_lyapunov_exponents_large = [v for (k, v) in lyapunov_exponents_large_gap] |> mean;

In [ ]:
avg_bogo_spec_small = [v for (k, v) in bogo_spec_small_gap] |> mean
avg_bogo_spec_large = [v for (k, v) in bogo_spec_large_gap] |> mean;

In [ ]:
lyapunov_exponents_small_gap |> length |> println
lyapunov_exponents_large_gap |> length |> println

In [ ]:
function moving_average(vs, n)
    res = similar(vs, length(vs) - (n-1))
    @inbounds for i in 1:length(res)
        res[i] = sum(@view vs[i:(i + n-1)]) / n
    end
    return res
end

navg = 32
ninterp = size(avg_lyapunov_exponents_small)[2] - 1
avg_lyapunov_exponents_small_smooth = linear_interpolation(range(0, 1, ninterp + 1)[1:end - navg + 1], moving_average(avg_lyapunov_exponents_small, navg), extrapolation_bc=Line());
avg_lyapunov_exponents_large_smooth = linear_interpolation(range(0, 1, ninterp + 1)[1:end - navg + 1], moving_average(avg_lyapunov_exponents_large, navg), extrapolation_bc=Line());

In [ ]:
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))
ylims = Dict(9 => 0.3, 11 => 0.4, 13 => 0.4, 15 => 0.5, 17 => 0.6, 19 => 0.7)

figure(figsize=(7, 3))
ax = subplot(121)
plot(range(0, 1, 33), avg_eigvals_small, "-o", lw=2, ms=4)
plot(range(0, 1, 33), avg_eigvals_large, "-", ms=3)
plot(range(0, 1, 33)[2:end], avg_bogo_spec_small[N, :], lw=2, "--C0")
plot(range(0, 1, 33)[2:end], avg_bogo_spec_large[N, :], "--C1")
plot([], [], "-k", label="Exact")
plot([], [], "--k", label="Bogoliubov")
xlim(0, 1)
ax.set_xticks([0., 0.5, 1.0])
ylim(0, 2)
xlabel("\$s\$")
ylabel("Average Minigap")
legend(frameon=false)

ax = subplot(122)
# plot(range(0, 1, npts+1), avg_lyapunov_exponents_small[1, :], "-", ms=3)
plot(range(0, 1, npts+1), map(x -> avg_lyapunov_exponents_small_smooth(x - navg / 2ninterp), range(0, 1, npts+1)), lw=2, label="Small")
# plot(range(0, 1, npts+1), avg_lyapunov_exponents_large[1, :], "--", label="Large")
plot(range(0, 1, npts+1), map(x -> avg_lyapunov_exponents_large_smooth(x - navg / 2ninterp), range(0, 1, npts+1)), label="Large")

xlim(0, 1)
ax.set_xticks([0., 0.5, 1.0])
ylim(0, ylims[N])
xlabel("\$s\$")
ylabel("Average \$\\mathrm{Tr}\\,\\bm{F}(t, t)\$")
legend(frameon=false)
tight_layout()
# savefig(PLOT_PATH * @sprintf("gap_flucs_N_%i.pdf", N), dpi=256, bbox_inches="tight")

In [ ]:
length(couplings_small_gap) |> println
length(couplings_large_gap) |> println

In [ ]:
gapsize_small_gap = []
gaplocs_small_gap = []
for (k, v) in eigvals_small_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_small_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_small_gap, gaploc)
end 

In [ ]:
gapsize_large_gap = []
gaplocs_large_gap = []
for (k, v) in eigvals_large_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_large_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_large_gap, gaploc)
end 

In [ ]:
figure(figsize=(7, 3))
ax = subplot(121)
hist2D(gapsize_small_gap, gaplocs_small_gap, bins=32, range=[[0, 1e-2], [0., 1.0]], cmap="gist_earth_r")
colorbar()
# colorbar(ticks=[0, 1, 2, 3])
xlim(0, 0.01)
ylim(0., 1)
xticks([0., 5e-3, 1e-2])
ax.set_xticklabels([0., "", 1e-2])
xlabel("\$\\Delta\$")
ylabel("\$s_{\\mathrm{min}}\$")

ax = subplot(122)
s_range = [0.5, 1.0]
# s_range = [0.4, 0.9]
hist2D(gapsize_large_gap, gaplocs_large_gap, bins=32, range=[s_range, [0., 1.0]], cmap="gist_earth_r")
colorbar()
# colorbar(ticks=collect(0:2:12))
xlim(s_range)
ylim(0.0, 1)
ax.set_yticklabels([])
xlabel("\$\\Delta\$")
# ylabel("\$s_{\\mathrm{min}}\$")
tight_layout(pad=0)
# savefig(PLOT_PATH * @sprintf("gap_loc_size_N_%i.pdf", N), dpi=256, bbox_inches="tight")

## Testing

In [ ]:
avg_most_undecided_flucs_small = [v for (k, v) in most_undecided_flucs_small] |> mean
avg_most_undecided_flucs_large = [v for (k, v) in most_undecided_flucs_large] |> mean;

avg_norm_most_undecided_flucs_small = [v ./ maximum(v) for (k, v) in most_undecided_flucs_small] |> mean;
# avg_most_undecided_flucs_large = [v for (k, v) in most_undecided_flucs_large] |> mean;

In [ ]:
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))
ylims = Dict(9 => 0.1, 11 => 0.4, 13 => 0.4, 15 => 0.5, 17 => 0.6, 19 => 0.7)

figure(figsize=(7, 3))
ax = subplot(121)
plot(range(0, 1, 33), avg_eigvals_small, "-o", lw=2, ms=4)
plot(range(0, 1, 33), avg_eigvals_large, "-", ms=3)
plot(range(0, 1, 33)[2:end], avg_bogo_spec_small[N, :], lw=2, "--C0")
plot(range(0, 1, 33)[2:end], avg_bogo_spec_large[N, :], "--C1")
plot([], [], "-k", label="Exact")
plot([], [], "--k", label="Bogoliubov")
xlim(0, 1)
ax.set_xticks([0., 0.5, 1.0])
ylim(0, 2)
xlabel("\$s\$")
ylabel("Average Minigap")
legend(frameon=false)

ax = subplot(122)
# plot(range(0, 1, npts+1), avg_most_undecided_flucs_small .- avg_lyapunov_exponents_small[1, :] ./ (N - 1), lw=2)
plot(range(0, 1, npts+1), avg_most_undecided_flucs_small ./ maximum(avg_most_undecided_flucs_small), lw=2)
plot(range(0, 1, npts+1), avg_norm_most_undecided_flucs_small)
# plot(range(0, 1, npts+1), avg_most_undecided_flucs_large, label="Large")

xlim(0, 1)
ax.set_xticks([0., 0.5, 1.0])
ylim(0, )
xlabel("\$s\$")
ylabel("Average \$\\mathrm{Tr}\\,\\bm{F}(t, t)\$")
legend(frameon=false)
tight_layout()

In [ ]:
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

figure(figsize=(4, 3))
subplot(111)
plot(range(0, 1, 33), avg_eigvals_small, "--k")
plot(range(0, 1, 33)[2:end], avg_bogo_spec_small[N, :], "-C0") 
plot(range(0, 1, 33), avg_eigvals_large, "-k", ms=2.5)
plot(range(0, 1, 33)[2:end], avg_bogo_spec_large[N, :], "-C1") 
xlim(0., 1.)
ylim(0., 2)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

In [ ]:
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

figure(figsize=(4, 3))
subplot(111)
for i in 1:size(avg_bogo_spec_large)[1]
    plot(range(0, 1, 33), avg_eigvals_small, "--k")
    plot(range(0, 1, 33)[2:end], avg_bogo_spec_small[i, :], "-C0") 
    plot(range(0, 1, 33), avg_eigvals_large, "-k", ms=2.5)
    plot(range(0, 1, 33)[2:end], avg_bogo_spec_large[i, :], "-C1") 
    
end
xlim(0., 1.)
ylim(0., 2)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

In [ ]:
# instance_seeds = []
# results_seeds = []

# for instance_name in instance_names
#     seed = match(pattern, instance_name)[1]
#     if occursin("results", instance_name)
#         push!(results_seeds, seed)        
#         continue
#     end
#     push!(instance_seeds, seed)        
# end

# instance_seeds = parse.(Int, instance_seeds)
# results_seeds = parse.(Int, results_seeds);